In [ ]:
import sys
import cv2
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
def plot_img(name, img):
    plt.imshow(img)
    plt.title(name)
    plt.show()

In [ ]:
def in_bounds(s, target):
    return (s/target >= 0.7 and s/target <= 1.3)

def digitGenerator(thresh, height_of_digit, width_of_digit):
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        [x, y, w, h] = cv2.boundingRect(cnt)
        if (in_bounds(w, width_of_digit) and in_bounds(h, height_of_digit)):
            cv2.rectangle(thresh, (x - 1, y - 1), (x + 1 + w, y + 1 + h), (0, 0, 255), 1)
            roi = thresh[y:y + h, x:x + w]
            roi = cv2.adaptiveThreshold(roi,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,21,11)
            yield roi
        else:
            print(w,h)

In [ ]:
im = cv2.imread("./courrier-font.png")
imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(imgray, 150, 255, 0)
plot_img('thresh', thresh)
width_of_digit = 58
height_of_digit = 84
            
digitsModels = [[None]] * 11
for roi in digitGenerator(thresh, height_of_digit, width_of_digit):
    cv2.imshow('Training: Enter digits displayed in the red rectangle!', roi)
    key = cv2.waitKey(0)
    keys = [i for i in range(48, 58)]
    keys.append(121)
    if key in keys:
        if key != 121: 
            digitsModels[key - 48] = roi
        else:
            digitsModels[10] = roi
cv2.destroyAllWindows()

In [ ]:
def detect_number(digit, candidate):
    dgt = digit
    if (candidate.shape != dgt.shape):
        dgt = cv2.resize(dgt, np.transpose(candidate).shape)
    dgt[np.where(dgt > 80)] = 255
    dgt[np.where(dgt <= 80)] = 0
    conv = np.multiply(dgt, candidate)
    return np.sum(np.sum(conv))

def recognize_digit(candidate):
    convs = [detect_number(digit, candidate) for digit in digitsModels]
    print(candidate.shape, convs)
    return convs.index(max(convs))

In [ ]:
recognize_digit(digitsModels[8])

In [ ]:
im = cv2.imread("./fapiao/croped/43614530.png")
imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(imgray, 120, 255, 0)
plot_img('thresh', thresh)
width_of_digit = 15
height_of_digit = 33

for roi in digitGenerator(thresh, height_of_digit, width_of_digit):
    cv2.imshow('Training: Enter digits displayed in the red rectangle!', roi)
    try:
        print("Digit is:", recognize_digit(roi))
    except Exception as e:
        print("Size Exception", e)
        cv2.destroyAllWindows()
        break
    cv2.waitKey(0)
cv2.destroyAllWindows()